<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 15.2 MB/s 
     |████████████████████████████████| 182 kB 79.2 MB/s 
     |████████████████████████████████| 168 kB 70.4 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 80.7 MB/s 
     |████████████████████████████████| 166 kB 79.7 MB/s 
     |████████████████████████████████| 166 kB 86.2 MB/s 
     |████████████████████████████████| 162 kB 85.5 MB/s 
     |████████████████████████████████| 162 kB 69.9 MB/s 
     |████████████████████████████████| 158 kB 69.0 MB/s 
     |████████████████████████████████| 157 kB 82.0 MB/s 
     |████████████████████████████████| 157 kB 80.6 MB/s 
     |████████████████████████████████| 157 kB 81.2 MB/s 
     |████████████████████████████████| 157 kB 86.2 MB/s 
     |████████████████████████████████| 157 kB 54.2 MB/s 
     |████████████████████████████████| 157 kB 87.1 MB/s 
     |████████████████████████████████| 157 kB 82.9 MB/s 
     |██████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
efficientnet = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
    )

model_efficientnet=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  efficientnet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
efficientnet.trainable=False
model_efficientnet.summary()

16705208/16705208 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 4,055,976
Trainable params: 6,405
Non-trainable params: 4,049,571
_________________________________________________________________


In [7]:
model_efficientnet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "EfficientnetB0"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_efficientnet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.4513 - accuracy: 0.3622

wandb: Adding directory to artifact (/content/wandb/run-20221213_110705-1x2t4mwv/files/model-best)... Done. 0.1s


22/22 [==============================] - 228s 7s/step - loss: 1.4513 - accuracy: 0.3622 - val_loss: 1.4410 - val_accuracy: 0.3886
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 1.4017 - accuracy: 0.3636

wandb: Adding directory to artifact (/content/wandb/run-20221213_110705-1x2t4mwv/files/model-best)... Done. 0.1s


...

22/22 [==============================] - 37s 2s/step - loss: 1.3816 - accuracy: 0.4006 - val_loss: 1.3818 - val_accuracy: 0.3886
Epoch 28/40
22/22 [==============================] - 3s 118ms/step - loss: 1.3791 - accuracy: 0.4190 - val_loss: 1.3960 - val_accuracy: 0.3886
Epoch 29/40
22/22 [==============================] - 3s 115ms/step - loss: 1.3751 - accuracy: 0.4219 - val_loss: 1.4077 - val_accuracy: 0.3886
Epoch 30/40
22/22 [==============================] - 3s 115ms/step - loss: 1.3987 - accuracy: 0.4162 - val_loss: 1.3825 - val_accuracy: 0.3886
Epoch 31/40
22/22 [==============================] - 3s 115ms/step - loss: 1.3735 - accuracy: 0.4148 - val_loss: 1.4036 - val_accuracy: 0.3886
Epoch 32/40
22/22 [==============================] - 3s 115ms/step - loss: 1.3676 - accuracy: 0.4247 - val_loss: 1.3980 - val_accuracy: 0.3886
Epoch 33/40
22/22 [==============================] - 3s 115ms/step - loss: 1.3791 - accuracy: 0.4134 - val_loss: 1.3877 - val_accuracy: 0.3886
Epoch 34/40
2

wandb: Adding directory to artifact (/content/wandb/run-20221213_110705-1x2t4mwv/files/model-best)... Done. 0.1s


22/22 [==============================] - 39s 2s/step - loss: 1.3782 - accuracy: 0.3991 - val_loss: 1.3816 - val_accuracy: 0.3886
Epoch 38/40
22/22 [==============================] - 3s 116ms/step - loss: 1.3767 - accuracy: 0.4190 - val_loss: 1.4081 - val_accuracy: 0.3886
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 1.3761 - accuracy: 0.4219

wandb: Adding directory to artifact (/content/wandb/run-20221213_110705-1x2t4mwv/files/model-best)... Done. 0.1s


22/22 [==============================] - 38s 2s/step - loss: 1.3761 - accuracy: 0.4219 - val_loss: 1.3787 - val_accuracy: 0.3886
Epoch 40/40
22/22 [==============================] - 3s 114ms/step - loss: 1.3798 - accuracy: 0.4290 - val_loss: 1.4018 - val_accuracy: 0.3886
CPU times: user 8min 57s, sys: 29.1 s, total: 9min 27s
Wall time: 13min 19s


In [10]:
wandb.alert(title = "Run finished",
            text = "efficientnetb0")

In [11]:
wandb.finish()

accuracy,▁▁▂▄▂▂▃▂▂▄▄▆▄▅▃▅▆▇▅▆▄▆█▄▆▂▅▇▇▆▆▇▆▇█▅▅▇▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▄▃▄▄▅▄▃▄▃▃▃▃▃▃▃▁▃▂▂▂▂▃▃▃▂▂▂▄▂▁▂▂▁▃▂▂▂▂
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁
val_loss,█▇▇▇▆▅▇▅▆▅▅▄▆▄▅▅▄▄▃▃▅▄▂▄▄▃▁▃▄▁▄▃▂▄▄▂▁▄▁▄
accuracy,0.42898
best_epoch,38
best_val_loss,1.37868
epoch,39
loss,1.37977
val_accuracy,0.38857


In [ ]:
!nvidia-smi --query-gpu=gpu_name --format=csv

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

